In [ ]:
%cd '/content/drive/MyDrive/8th-semester/IR/IR_Project_W23/src'

/content/drive/MyDrive/8th-semester/IR/IR_Project_W23/src


In [ ]:
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 57.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 50.8 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=3854dbc9133865e65292d63c76480c6300d79e4654ebf70fcd5578224e977ed2
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence-transformers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version th

In [ ]:
!pip install faiss-cpu
!pip install faiss-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 42.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.3 MB/s eta 0:00:00


### Import Libraries

In [ ]:
import os
import sys
import requests
import json
import pandas as pd
import numpy as np
from tqdm import tqdm, trange
import torch
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import pickle
from pathlib import Path
from vector_engine.vector_engine.utils import vector_search, id2details

### Loading Data

In [ ]:
# read data from the csv file (from the location it is stored)
data_dir = '../data/'
data_list = os.listdir(data_dir)
dataframe_list = []
print(data_list)

for data_file in data_list:
    if '.json' not in data_file:
        continue
    data_path = data_dir + data_file
    data_name = data_file[:-5]
    save_data_path = data_dir + 'summary/' + data_name + '.xlsx'
    print("Data File:", data_name)
    df = pd.read_json(data_path)
    df = df.astype(str)
    rows, columns = df.shape
    dataframe_list.append(df)

df = pd.concat(dataframe_list)
df.head(20)

['summary', 'LSTM_Attention', 'backup', 'cnn10.json', 'cnn10_full.json', 'ramsay10.json', 'ted.json', 'reviews.json', 'podcasts.json']
Data File: cnn10
Data File: cnn10_full
Data File: ramsay10
Data File: ted
Data File: reviews
Data File: podcasts


,id,text_captions,fused_captions,summary,snippet,statistics
0,8GgaCgjAFLM,[music] Thank you Hello lovely people Happy! F...,"[{'text': '[Music]', 'start': 0.0, 'duration':...",It's been nearly three years since the start ...,"{'publishedAt': '2023-03-02T01:20:33Z', 'chann...","{'viewCount': '104381', 'likeCount': '1046', '..."
1,Kc_GMYHJNFQ,[music] Thank you hello everyone Wonderful! We...,"[{'text': '[Music]', 'start': 0.0, 'duration':...","Some farmers are not harvesting any tomatoes,...","{'publishedAt': '2023-02-28T23:25:02Z', 'chann...","{'viewCount': '97761', 'likeCount': '1049', 'f..."
2,wPdiuj2JwEU,[music] Thank you Hey there everyone Terrific!...,"[{'text': '[Music]', 'start': 0.0, 'duration':...",CNN Business Correspondent Rahel Solomon has ...,"{'publishedAt': '2023-02-27T22:44:53Z', 'chann...","{'viewCount': '100789', 'likeCount': '1168', '..."
3,H07TJT3VpSY,"[music] Thank you! What's up Sunshine, Welcome...","[{'text': '[Music]', 'start': 0.0, 'duration':...",Friday marked one year since Russian Presiden...,"{'publishedAt': '2023-02-26T19:36:20Z', 'chann...","{'viewCount': '84078', 'likeCount': '759', 'fa..."
4,U4WxR7hXfCQ,[music] Thank you! What's up lovely people? Th...,"[{'text': '[Music]', 'start': 0.0, 'duration':...",Massive storm system pummeling much of the co...,"{'publishedAt': '2023-02-24T00:34:44Z', 'chann...","{'viewCount': '90645', 'likeCount': '990', 'fa..."
5,1xbEbTJASoE,[music] Thank you What's up lovely people! It'...,"[{'text': '[Music]', 'start': 0.0, 'duration':...",A massive winter storm is threatening the U.S...,"{'publishedAt': '2023-02-23T00:50:47Z', 'chann...","{'viewCount': '82775', 'likeCount': '1056', 'f..."
6,rQp4putiL_A,[music] Thank you Hello lovely people! Wonderf...,"[{'text': '[Music]', 'start': 0.0, 'duration':...",CNN's Brian Todd tells us about President Bid...,"{'publishedAt': '2023-02-22T00:41:45Z', 'chann...","{'viewCount': '77883', 'likeCount': '907', 'fa..."
7,rQp4putiL_A,[music] Thank you Hello lovely people! Wonderf...,"[{'text': '[Music]', 'start': 0.0, 'duration':...",CNN's Brian Todd tells us about President Bid...,"{'publishedAt': '2023-02-22T00:41:45Z', 'chann...","{'viewCount': '77883', 'likeCount': '907', 'fa..."
8,94N_3jBAU_c,[music] Thank you hello everyone! Terrific! Tu...,"[{'text': '[Music]', 'start': 0.0, 'duration':...",This week President Biden made an unexpected ...,"{'publishedAt': '2023-02-21T14:03:38Z', 'chann...","{'viewCount': '98478', 'likeCount': '807', 'fa..."
9,owDr-ra0P6I,"[music] Thank you hello lovely people, You mad...","[{'text': '[Music]', 'start': 0.0, 'duration':...",A toxic train derailment in Ohio led to the e...,"{'publishedAt': '2023-02-17T00:32:51Z', 'chann...","{'viewCount': '95155', 'likeCount': '1130', 'f..."


In [ ]:
df.describe()

,id,text_captions,fused_captions,summary,snippet,statistics
count,445,445,445,445,445,445
unique,435,437,435,436,435,435
top,rQp4putiL_A,[music] Thank you Hello lovely people Happy! F...,"[{'text': '[Music]', 'start': 0.0, 'duration':...",CNN's Brian Todd tells us about President Bid...,"{'publishedAt': '2023-02-22T00:41:45Z', 'chann...","{'viewCount': '77883', 'likeCount': '907', 'fa..."
freq,3,2,3,3,3,3


In [ ]:
print(f"Videos: {df.id.unique().shape[0]}")
n_ids = len(df.id.unique())
id_list = df.id.unique()
id_value_list = np.arange(n_ids)
id_dict = dict(zip(id_list, id_value_list))
# id_dict = {id_list[i]: id_value_list[i] for i in range(n_ids)}
df['id_value'] = df['id'].apply(lambda x: id_dict[x])
df.head()

Videos: 435


,id,text_captions,fused_captions,summary,snippet,statistics,id_value
0,8GgaCgjAFLM,[music] Thank you Hello lovely people Happy! F...,"[{'text': '[Music]', 'start': 0.0, 'duration':...",It's been nearly three years since the start ...,"{'publishedAt': '2023-03-02T01:20:33Z', 'chann...","{'viewCount': '104381', 'likeCount': '1046', '...",0
1,Kc_GMYHJNFQ,[music] Thank you hello everyone Wonderful! We...,"[{'text': '[Music]', 'start': 0.0, 'duration':...","Some farmers are not harvesting any tomatoes,...","{'publishedAt': '2023-02-28T23:25:02Z', 'chann...","{'viewCount': '97761', 'likeCount': '1049', 'f...",1
2,wPdiuj2JwEU,[music] Thank you Hey there everyone Terrific!...,"[{'text': '[Music]', 'start': 0.0, 'duration':...",CNN Business Correspondent Rahel Solomon has ...,"{'publishedAt': '2023-02-27T22:44:53Z', 'chann...","{'viewCount': '100789', 'likeCount': '1168', '...",2
3,H07TJT3VpSY,"[music] Thank you! What's up Sunshine, Welcome...","[{'text': '[Music]', 'start': 0.0, 'duration':...",Friday marked one year since Russian Presiden...,"{'publishedAt': '2023-02-26T19:36:20Z', 'chann...","{'viewCount': '84078', 'likeCount': '759', 'fa...",3
4,U4WxR7hXfCQ,[music] Thank you! What's up lovely people? Th...,"[{'text': '[Music]', 'start': 0.0, 'duration':...",Massive storm system pummeling much of the co...,"{'publishedAt': '2023-02-24T00:34:44Z', 'chann...","{'viewCount': '90645', 'likeCount': '990', 'fa...",4


In [ ]:
# Instantiate the sentence-level DistilBERT
# model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')
model = SentenceTransformer('all-mpnet-base-v2')
# Check if GPU is available and use it
if torch.cuda.is_available():
    model = model.to(torch.device("cuda"))
print(model.device)

cuda:0


In [ ]:
# Convert summaries to vectors
embeddings = model.encode(df.summary.to_list(), show_progress_bar=True)

Batches:   0%|          | 0/14 [00:00<?, ?it/s]

In [ ]:
print(f'Shape of the summary vector: {embeddings[0].shape, type(embeddings)}')

Shape of the summary vector: ((768,), <class 'numpy.ndarray'>)


In [ ]:
# Step 1: Change data type
embeddings = np.array([embedding for embedding in embeddings]).astype("float32")

# Step 2: Instantiate the index
index = faiss.IndexFlatL2(embeddings.shape[1])

# Step 3: Pass the index to IndexIDMap
index = faiss.IndexIDMap(index)

# Step 4: Add vectors and their IDs
index.add_with_ids(embeddings, df.id_value.values)

print(f"Number of vectors in the Faiss index: {index.ntotal}")

Number of vectors in the Faiss index: 445


### Searching the index

In [ ]:
def id2details(df, I, column):
    """Returns the video details based on the video id."""
    return [list(df[df.id_value == idx][column]) for idx in I[0]]

In [ ]:
df['summary'].iloc[15]
# Retrieve the 10 nearest neighbours
D, I = index.search(np.array([embeddings[15]]), k=10)
print(f'L2 distance: {D.flatten().tolist()}\n\nMAG video IDs: {I.flatten().tolist()}')

L2 distance: [0.0, 0.0, 0.6978519558906555, 0.8730857372283936, 0.9603878259658813, 1.056288242340088, 1.0606818199157715, 1.061436414718628, 1.0672942399978638, 1.0715246200561523]

MAG video IDs: [5, 5, 17, 13, 55, 41, 23, 84, 39, 36]


In [ ]:
# Fetch the video based on their index
print(id2details(df, I, 'id_value'))
print(id2details(df, I, 'summary'))

[[5, 5], [5, 5], [17], [13], [55], [41], [23], [84], [39], [36]]
[[" A massive winter storm is threatening the U.S. Coast to Coast all the way from California and the West to Maine in the east. CNN's Miguel Marquez is in East Palestine with more a toxic mess left by its Durrell train now President Joe Biden is adding his pressure on Norfolk Southern following criticism for his response to the crash on Instagram. The EPA has ordered Norfolk Southern Rail Company to cover all of the cleanup costs but the rail company is facing further pressure. China's foreign Ministry calls the whole thing an accident. The U.S. says this was no weather balloon deflating. China is sticking to its weather balloon claim dismissed by the Pentagon as nothing but hot air. CNN's searched for the truth behind the balloon took our team from the Chinese Capital to Shanxi Province about 300 miles Southwest of Beijing.", " A massive winter storm is threatening the U.S. Coast to Coast all the way from California and

### User Query

In [ ]:
user_query = """massive winter storm in the united states"""
# For convenience, I've wrapped all steps in the vector_search function.
# It takes four arguments: 
# A query, the sentence-level transformer, the Faiss index and the number of requested results
D, I = vector_search([user_query], model, index, num_results=10)
print(f'L2 distance: {D.flatten().tolist()}\n\nMAG video IDs: {I.flatten().tolist()}')

L2 distance: [1.0660897493362427, 1.0660897493362427, 1.0968725681304932, 1.1540255546569824, 1.1702733039855957, 1.2011408805847168, 1.2462339401245117, 1.25309157371521, 1.3073045015335083, 1.3092975616455078]

MAG video IDs: [5, 5, 82, 17, 39, 47, 34, 18, 74, 7]


In [ ]:
# Fetching the paper titles based on their index
id2details(df, I, 'id')

[['1xbEbTJASoE', '1xbEbTJASoE'],
 ['1xbEbTJASoE', '1xbEbTJASoE'],
 ['8TqtknjkJQg'],
 ['CMrXA2Qyi3A'],
 ['AMJigDmp3Ik'],
 ['08IAa4cCZ5U'],
 ['ouF8jUAucGw'],
 ['F06fMTGmXyg'],
 ['m3yjp4yvx9w'],
 ['94N_3jBAU_c', '94N_3jBAU_c']]

In [ ]:
project_dir = '/content/drive/MyDrive/8th-semester/IR/IR_Project_W23'
# Serialise index and store it as a pickle
with open(f"{project_dir}/models/faiss_index.pickle", "wb") as h:
    pickle.dump(faiss.serialize_index(index), h)